## Insert Parameters

In [2]:
%matplotlib inline
import site, os, glob, pandas
site.addsitedir(r'C:/Users/Ciaran/Documents/pycotools')
site.addsitedir(r'/home/b3053674/Documents/pycotools')
from pycotools import model, tasks, viz, misc

/home/b3053674/Documents/pycotools/pycotools/viz.py:117: FutureWarning: The pandas.parser module is deprecated and will be removed in a future version. Please import from the pandas.io.parser instead
  from pandas.parser import CParserError


### Build Example Model
Here we use the same model as in the QuickStart tutorial.

In [3]:
working_directory = r'C:\Users\Ciaran\Documents\PycotoolsQuickStart'
working_directory = r'/home/b3053674/Documents/Models/2017/12_Dec/PycotoolsWorkflowDir'
copasi_file = os.path.join(working_directory, 'MichaelisMenten.cps')

if os.path.isfile(copasi_file):
    os.remove(copasi_file)

    
kf = 0.01
kb = 0.1
kcat = 0.05
with model.Build(copasi_file) as m:
    m.name = 'Michaelis-Menten'
    m.add('compartment', name='Cell')
    
    m.add('metabolite', name='P', concentration=0)
    m.add('metabolite', name='S', concentration=30)
    m.add('metabolite', name='E', concentration=10)
    m.add('metabolite', name='ES', concentration=0)
    
    m.add('reaction', name='S bind E', expression='S + E -> ES', rate_law='kf*S*E',
          parameter_values={'kf': kf})
    
    m.add('reaction', name='S unbind E', expression='ES -> S + E', rate_law='kb*ES',
         parameter_values={'kb': kb})

    m.add('reaction', name='ES produce P', expression='ES -> P + E', rate_law='kcat*ES', 
          parameter_values={'kcat': kcat})
    
michaelis_menten = model.Model(copasi_file)
michaelis_menten

Model(name=Michaelis-Menten, time_unit=s, volume_unit=ml, quantity_unit=mmol)

### Insert Parameters from Python Dictionary

In [8]:
params = {'E': 100,
          'P': 150}

## Insert into model 
I = model.InsertParameters(michaelis_menten, parameter_dict=params)
print(I.parameters.transpose())

## Save the model
I.model.save()

     0
E  100
P  150


Model(name=Michaelis-Menten, time_unit=s, volume_unit=ml, quantity_unit=mmol)

Alternatively use `inplace=True` argument to save automatically

In [ ]:
I = model.InsertParameters(michaelis_menten, parameter_dict=param, inplace=True)

### Insert Parameters from Pandas DataFrame

In [10]:
params = {'(S bind E).kf': 50,
          '(S unbind E).kb': 96}
df = pandas.DataFrame(params, index=[0])

model.InsertParameters(michaelis_menten, df=df, inplace=True)

### Insert Parameters from Parameter Estimation Output
Get some parameter estimation data

In [12]:
## simulate data 
TC = tasks.TimeCourse(michaelis_menten, end=100, step_size=1, intervals=100)

## format copasi report headers
misc.format_timecourse_data(TC.report_name)

## run PEs
PE = tasks.MultiParameterEstimation(michaelis_menten, TC.report_name, 
                                   method='genetic_algorithm', 
                                   population_size=10, 
                                   number_of_generations=10, 
                                   copy_number=2, 
                                   pe_number=2, 
                                   run_mode=True)
PE.write_config_file()
PE.setup()
PE.run()
print (PE.results_directory)

pycotools.tasks:INFO:3661:    running model: 0
pycotools.tasks:INFO:3661:    running model: 1
/home/b3053674/Documents/Models/2017/12_Dec/PycotoolsWorkflowDir/MultipleParameterEstimationResults


In [13]:
##index=0 for best parameter set (i.e. lowest RSS)
model.InsertParameters(michaelis_menten, parameter_path=PE.results_directory, index=0, inplace=True) 

### Insert Parameters using the `model.Model().insert_parameters` method
The same means of inserting parameters can be used from the model object itself

In [ ]:
michaelis_menten.insert_parameters(parameter_dict=params, inplace=True)

### Change parameters using `model.Model().set`
Individual parameters can also be changed using the `set` method. For example, we could set the `metabolite` with name `S` `concentration` or `particle numbers` to `55`

In [ ]:
michaelis_menten.set('metabolite', 'S', 55, 'name', 'concentration')

## or 

michaelis_menten.set('metabolite', 'S', 55, 'name', 'particle_numbers')